### Processamento dos dados do enade a fim de facilitar a construção da visualização
A visualização pode ser vista em: https://beta.observablehq.com/@lorenaps/como-anda-o-incentivo-e-o-desempenho-no-ensino-superior-na-pa

O objetivo é preparar os dados para responder as seguintes perguntas:
- Existe diferença de renda entre os alunos que tiveram acesso ao ensino superior através de políticas de ação afirmativa e aqueles que obtiveram acesso sem a utilização de políticas afirmativas?
- Qual a escolaridade dos pais entre os estudantes via política afirmativa e acesso normal?
- De onde vem o incentivo para cursar a graduação?
- Eles são os primeiros de sua família a concluir o ensino superior?

Mais informações sobre os dados base podem ser encontradas em: http://portal.inep.gov.br/web/guest/microdados

In [6]:
import pandas as pd
import io
import requests

In [7]:
url = 'https://raw.githubusercontent.com/nazareno/enade-vis/master/data/enade_2017_pb-str.csv'
dados_url = requests.get(url).content

In [8]:
enade = pd.read_csv(io.StringIO(dados_url.decode('utf-8')))
enade.head()

,id,A biblioteca dispôs das referências bibliográficas que os estudantes necessitaram.,A coordenação do curso esteve disponível para orientação acadêmica dos estudantes.,A fundamentação teórica oferecida no curso de Licenciatura foi suficiente para sua compreensão sobre a educação escolar e sua preparação para o exercício da docência?,A instituição contou com biblioteca virtual ou conferiu acesso a obras disponíveis em acervos virtuais.,A instituição dispôs de quantidade suficiente de funcionários para o apoio administrativo e acadêmico.,"A instituição dispôs de refeitório, cantina e banheiros em condições adequadas que atenderam as necessidades dos seus usuários.",A instituição ofereceu oportunidades para os estudantes atuarem como representantes em órgãos colegiados.,"A instituição promoveu atividades de cultura, de lazer e de interação social.",Alguém em sua família concluiu um curso superior?,...,Instituição(IES),Sigla,Endereço,Município,UF,Organização Acadêmica,Categoria,Categoria Administrativa,IGC,Ano IGC
0,2,6.0,6.0,"Sim, completamente",6.0,6.0,6.0,6.0,6.0,Sim,...,FACULDADES INTEGRADAS DE PATOS (FIP),FIP,Rua Horácio Nóbrega - s/n - s/c - CEP: 58704-0...,Patos,PB,Faculdade,Privada,Privada com fins lucrativos,3,2016
1,3,6.0,6.0,"Sim, completamente",6.0,6.0,6.0,6.0,6.0,Não,...,FACULDADES INTEGRADAS DE PATOS (FIP),FIP,Rua Horácio Nóbrega - s/n - s/c - CEP: 58704-0...,Patos,PB,Faculdade,Privada,Privada com fins lucrativos,3,2016
2,4,6.0,6.0,"Sim, completamente",6.0,6.0,6.0,3.0,6.0,Não,...,FACULDADES INTEGRADAS DE PATOS (FIP),FIP,Rua Horácio Nóbrega - s/n - s/c - CEP: 58704-0...,Patos,PB,Faculdade,Privada,Privada com fins lucrativos,3,2016
3,5,3.0,6.0,"Sim, em grande parte",4.0,6.0,6.0,1.0,1.0,Não,...,FACULDADES INTEGRADAS DE PATOS (FIP),FIP,Rua Horácio Nóbrega - s/n - s/c - CEP: 58704-0...,Patos,PB,Faculdade,Privada,Privada com fins lucrativos,3,2016
4,6,6.0,6.0,"Sim, completamente",4.0,6.0,5.0,6.0,6.0,Sim,...,FACULDADES INTEGRADAS DE PATOS (FIP),FIP,Rua Horácio Nóbrega - s/n - s/c - CEP: 58704-0...,Patos,PB,Faculdade,Privada,Privada com fins lucrativos,3,2016


In [9]:
enade.columns.values

array(['id',
       'A biblioteca dispôs das referências bibliográficas que os estudantes necessitaram.',
       'A coordenação do curso esteve disponível para orientação acadêmica dos estudantes.',
       'A fundamentação teórica oferecida no curso de Licenciatura foi suficiente para sua compreensão sobre a educação escolar e sua preparação para o exercício da docência?',
       'A instituição contou com biblioteca virtual ou conferiu acesso a obras disponíveis em acervos virtuais.',
       'A instituição dispôs de quantidade suficiente de funcionários para o apoio administrativo e acadêmico.',
       'A instituição dispôs de refeitório, cantina e banheiros em condições adequadas que atenderam as necessidades dos seus usuários.',
       'A instituição ofereceu oportunidades para os estudantes atuarem como representantes em órgãos colegiados.',
       'A instituição promoveu atividades de cultura, de lazer e de interação social.',
       'Alguém em sua família concluiu um curso superio

In [10]:
enade.groupby("Qual a renda total de sua família, incluindo seus rendimentos?").size()

Qual a renda total de sua família, incluindo seus rendimentos?
Acima de 30 salários mínimos (mais de R$ 28110,00).           34
Até 1,5 salário mínimo (até R$ 1405,50).                    2561
De 1,5 a 3 salários mínimos (R$ 1405,51 a R$ 2.811,00).     2025
De 10 a 30 salários mínimos (R$ 9370,01 a R$ 28.110,00).     318
De 3 a 4,5 salários mínimos (R$ 2811,01 a R$ 4.216,50).     1000
De 4,5 a 6 salários mínimos (R$ 4216,51 a R$ 5.622,00).      433
De 6 a 10 salários mínimos (R$ 5 622,01 a R$ 9.370,00).      488
dtype: int64

Desconsiderando as diferentes classificações de politicas afirmativas

In [11]:
enade = enad.replace(['Sim, por critério de renda', 'Sim, por critério étnico-racial', 
                                'Sim, por sistema diferente dos anteriores', 
                                'Sim, por sistema que combina dois ou mais critérios anteriores', 
                                'Sim, por ter estudado em escola pública ou particular com bolsa de estudos'], 'Sim')

enade.groupby('Seu ingresso no curso de graduação se deu por meio de políticas de ação afirmativa ou inclusão social?').size().reset_index(name='qtd').sort_values('qtd', ascending=False)

,Seu ingresso no curso de graduação se deu por meio de políticas de ação afirmativa ou inclusão social?,qtd
0,Não,4960
1,Sim,1899


In [12]:
enade.groupby('Qual a renda total de sua família, incluindo seus rendimentos?').size().reset_index(name='qtd').sort_values('qtd', ascending=False)

,"Qual a renda total de sua família, incluindo seus rendimentos?",qtd
1,"Até 1,5 salário mínimo (até R$ 1405,50).",2561
2,"De 1,5 a 3 salários mínimos (R$ 1405,51 a R$ 2...",2025
4,"De 3 a 4,5 salários mínimos (R$ 2811,01 a R$ 4...",1000
6,"De 6 a 10 salários mínimos (R$ 5 622,01 a R$ 9...",488
5,"De 4,5 a 6 salários mínimos (R$ 4216,51 a R$ 5...",433
3,"De 10 a 30 salários mínimos (R$ 9370,01 a R$ 2...",318
0,Acima de 30 salários mínimos (mais de R$ 28110...,34


In [13]:
enade.groupby('Quem lhe deu maior incentivo para cursar a graduação?').size().reset_index(name='qtd').sort_values('qtd', ascending=False)

,Quem lhe deu maior incentivo para cursar a graduação?,qtd
5,Pais,4517
2,Ninguém,1082
4,Outros membros da família que não os pais,465
0,Colegas/Amigos,314
6,Professores,297
3,Outras pessoas,162
1,Líder ou representante religioso,22


In [14]:
enade.groupby('Alguém em sua família concluiu um curso superior?').size().reset_index(name='qtd').sort_values('qtd', ascending=False)  

,Alguém em sua família concluiu um curso superior?,qtd
1,Sim,4739
0,Não,2120


In [15]:
enade.groupby('Até que etapa de escolarização seu pai concluiu?').size().reset_index(name='qtd').sort_values('qtd', ascending=False)  

,Até que etapa de escolarização seu pai concluiu?,qtd
0,Ensino Fundamental: 1º ao 5º ano (1ª a 4ª série),2054
2,Ensino Médio,1815
3,Ensino Superior - Graduação,1010
4,Nenhuma,855
1,Ensino Fundamental: 6º ao 9º ano (5ª a 8ª série),818
5,Pós-graduação,307


In [16]:
enade.groupby('Até que etapa de escolarização sua mãe concluiu?').size().reset_index(name='qtd').sort_values('qtd', ascending=False)  

,Até que etapa de escolarização sua mãe concluiu?,qtd
3,Ensino médio,1988
1,Ensino fundamental: 1º ao 5º ano (1ª a 4ª série),1788
0,Ensino Superior - Graduação,1252
2,Ensino fundamental: 6º ao 9º ano (5ª a 8ª série),829
5,Pós-graduação,604
4,Nenhuma,398


In [17]:
enade.loc[(enad["Seu ingresso no curso de graduação se deu por meio de políticas de ação afirmativa ou inclusão social?"] == 'Sim') & 
         (enad["Qual a renda total de sua família, incluindo seus rendimentos?"] == 'Acima de 30 salários mínimos (mais de R$ 28110,00).')]

,id,A biblioteca dispôs das referências bibliográficas que os estudantes necessitaram.,A coordenação do curso esteve disponível para orientação acadêmica dos estudantes.,A fundamentação teórica oferecida no curso de Licenciatura foi suficiente para sua compreensão sobre a educação escolar e sua preparação para o exercício da docência?,A instituição contou com biblioteca virtual ou conferiu acesso a obras disponíveis em acervos virtuais.,A instituição dispôs de quantidade suficiente de funcionários para o apoio administrativo e acadêmico.,"A instituição dispôs de refeitório, cantina e banheiros em condições adequadas que atenderam as necessidades dos seus usuários.",A instituição ofereceu oportunidades para os estudantes atuarem como representantes em órgãos colegiados.,"A instituição promoveu atividades de cultura, de lazer e de interação social.",Alguém em sua família concluiu um curso superior?,...,Instituição(IES),Sigla,Endereço,Município,UF,Organização Acadêmica,Categoria,Categoria Administrativa,IGC,Ano IGC


In [18]:
dados_agrupados = enade.groupby(['Seu ingresso no curso de graduação se deu por meio de políticas de ação afirmativa ou inclusão social?', 
              'Qual a renda total de sua família, incluindo seus rendimentos?', 
              'Quem lhe deu maior incentivo para cursar a graduação?', 
              'Alguém em sua família concluiu um curso superior?',
              'Até que etapa de escolarização sua mãe concluiu?',
              'Até que etapa de escolarização seu pai concluiu?']).size().reset_index(name='qtd').sort_values('qtd', ascending=False)
dados_agrupados.head()

,Seu ingresso no curso de graduação se deu por meio de políticas de ação afirmativa ou inclusão social?,"Qual a renda total de sua família, incluindo seus rendimentos?",Quem lhe deu maior incentivo para cursar a graduação?,Alguém em sua família concluiu um curso superior?,Até que etapa de escolarização sua mãe concluiu?,Até que etapa de escolarização seu pai concluiu?,qtd
142,Não,"Até 1,5 salário mínimo (até R$ 1405,50).",Pais,Não,Ensino fundamental: 1º ao 5º ano (1ª a 4ª série),Ensino Fundamental: 1º ao 5º ano (1ª a 4ª série),99
381,Não,"De 1,5 a 3 salários mínimos (R$ 1405,51 a R$ 2...",Pais,Sim,Ensino médio,Ensino Médio,96
895,Sim,"Até 1,5 salário mínimo (até R$ 1405,50).",Pais,Não,Ensino fundamental: 1º ao 5º ano (1ª a 4ª série),Ensino Fundamental: 1º ao 5º ano (1ª a 4ª série),84
764,Não,"De 6 a 10 salários mínimos (R$ 5 622,01 a R$ 9...",Pais,Sim,Ensino Superior - Graduação,Ensino Superior - Graduação,81
596,Não,"De 3 a 4,5 salários mínimos (R$ 2811,01 a R$ 4...",Pais,Sim,Ensino médio,Ensino Médio,81


In [19]:
dados_agrupados.count()

Seu ingresso no curso de graduação se deu por meio de políticas de ação afirmativa ou inclusão social?    1281
Qual a renda total de sua família, incluindo seus rendimentos?                                            1281
Quem lhe deu maior incentivo para cursar a graduação?                                                     1281
Alguém em sua família concluiu um curso superior?                                                         1281
Até que etapa de escolarização sua mãe concluiu?                                                          1281
Até que etapa de escolarização seu pai concluiu?                                                          1281
qtd                                                                                                       1281
dtype: int64

In [20]:
from google.colab import files

dados_agrupados.to_csv('dados_agrupados.csv')
files.download('dados_agrupados.csv')

ModuleNotFoundError: No module named 'google'

In [21]:
dados_agrupados.to_csv('../dados/dados_enade.csv')